<a href="https://colab.research.google.com/github/francoishcm/zipline/blob/master/Zipline_Reloaded_BacktestEngine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash
wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
tar -xzf ta-lib-0.4.0-src.tar.gz
cd ta-lib/
./configure
make
make install

In [ ]:
!pip install sqlite3

In [ ]:
import zipline
zipline.__version__

In [ ]:
!zipline bundles

In [ ]:
from google.colab import drive #Google Drive

#Data Processing
import numpy as np 
import pandas as pd
from numpy import sort

#Data Connection
import sqlite3
import sqlalchemy as db


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip3 install quandl
import quandl
quandl.ApiConfig.api_key = "WDDHaLh3eG6vrEgiCCqy"
!QUANDL_API_KEY= zipline ingest -b quandl

In [ ]:
data = quandl.get("WIKI/AAPL")

In [ ]:
%matplotlib inline

from zipline import run_algorithm
from zipline.api import order_target_percent, symbol

from datetime import datetime

import pytz
import matplotlib.pyplot as plt

def intitialize (context):
    context.stock = symbol ('AAPL')
    context.index_average_window = 100

def handle_data(context, data):
    # Request history for the stock
    equities_hist = data.history(context.stock, "close", 
                                 context.index_average_window, "1d")

    if equities_hist[-1] > equities_hist.mean():
        stock_weight = 1.0
    else:
        stock_weight = 0.0

    order_target_percent(context.stock,stock_weight)

def analyze (context,perf):
    fig = plt.figure(figsize=(12,8))

    ax = fig.add_subplot(311)
    ax.set_title('Helmie Capital Results')
    ax.semilogy(perf['portfolio_value'],linestyle='-',label='Equity Curve', linewidth=3.0)
    ax.legend()
    ax.grid(False)

    ax = fig.add_subplot(312)
    ax.plot(perf['gross_leverage'],label='Exposure',linestyle='-',linewidth=1.0)
    ax.legend()
    ax.grid(True)

    ax = fig.add_subplot(313)
    ax.plot(perf['returns'],label='Returns',linestyle='-',linewidth=1.0)
    ax.legend()
    ax.grid(True)

start_date = datetime(1996,1,1, tzinfo=pytz.UTC)
end_date = datetime(2000,12,31, tzinfo=pytz.UTC)

results =run_algorithm(
    start=start_date,
    end=end_date,
    initialize=intitialize,
    analyze=analyze,
    handle_data=handle_data,
    capital_base=10000,
    data_frequency = 'daily', bundle='quandl'
)

# New Section